In [ ]:
! git clone https://github.com/LEYADEV/Vocabulary-Transfer.git

Cloning into 'Vocabulary-Transfer'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 18 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [ ]:
# Confirm GPU availability
# Runtime>Change runtime type
!nvidia-smi

Tue Jun 21 11:15:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install transformers
! pip install simpletransformers
! pip install sentencepiece

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%writefile step1.py
##### SCRIPT STARTS HERE #####
#!usr/bin/bash python
#step 1 - Create SentencePiece vocabulary for dataset
import sentencepiece as spm
import argparse


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--dataset", type=str, required=True)
    parser.add_argument("--prefix", type=str, required=True)
    parser.add_argument("--vocab_size", type=int, required=True)
    return parser.parse_args()


def create_vocab(dataset_pth, prefix, vocab_size):
    """output: {prefix}.model {prefix}.vocab"""
    print(dataset_pth, prefix, vocab_size)
    spm.SentencePieceTrainer.train(
        f"--input={dataset_pth} --model_prefix={prefix}  --vocab_size={vocab_size} \
        --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3 \
        --pad_piece=[PAD] --unk_piece=[UNK] --bos_piece=[BOS] --eos_piece=[EOS] \
        --user_defined_symbols=[CLS],[SEP],[MASK]"
    )


def main():
    args = parse_args()
    create_vocab(args.dataset, args.prefix, args.vocab_size)


if __name__ == "__main__":
    main()


Writing step1.py


In [ ]:
!python3 step1.py --dataset=/content/gdrive/Shareddrives/Gutenberg/MT/scripture/oro-HARX_2022_01_29.txt --prefix=oro-HARX --vocab_size=8000

In [ ]:
!python3 step1.py --dataset=/content/gdrive/Shareddrives/Gutenberg/MT/scripture/ko-BTKX_2021_10_14.txt --prefix=ko-BTKX --vocab_size=8000

In [ ]:
!python3 step1.py --dataset=/content/gdrive/Shareddrives/Gutenberg/MT/corpora/en-CCMatrix_en_ko_clean_5k.txt --prefix=en-CCMatrix --vocab_size=5000

In [ ]:
!python3 step1.py --dataset=/content/gdrive/Shareddrives/Gutenberg/MT/corpora/ko-CCMatrix_en_ko_clean_5k.txt --prefix=ko-CCMatrix --vocab_size=5000

In [ ]:
# add sentencepiece model from building arb-en mc4 dataset; for the .vocab file (Not working)
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.load('t5_small_spiece.model')

Create Test and Val sets


In [ ]:
!pip install python-dotenv
!pip install sacrebleu
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import sacrebleu

# Tell the SIL NLP tools where to find the Gutenberg resources
os.environ['SIL_NLP_DATA_PATH'] = "/content/gdrive/Shareddrives/Gutenberg"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 92 kB 7.9 MB/s 


In [ ]:
#uncomment to prepare df for trg->src also
import os
import pandas
def prepare_translation_datasets(data_path, src_lang, src, trg_lang, trg, test_size):
    src_text = []
    trg_text = []
    with open(os.path.join(data_path, src), "r", encoding="utf-8") as s, \
         open(os.path.join(data_path, trg), "r", encoding="utf-8") as t:
        for src_line, trg_line in zip(s, t):
            src_line = src_line.strip()
            trg_line = trg_line.strip()
            if len(src_line) == 0 or len(trg_line) == 0:
                continue
            src_text.append(src_line)
            trg_text.append(trg_line)

    X_train, X_test, y_train, y_test = train_test_split(src_text, trg_text, test_size=test_size)


    data = []
    for src, trg in zip(X_train, y_train):
        data.append([f"translate {src_lang} to {trg_lang}", src, trg])
        #data.append([f"translate {trg_lang} to {src_lang}", trg, src])

    train_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])


    data = []
    for src, trg in zip(X_test, y_test):
        data.append([f"translate {src_lang} to {trg_lang}", src, trg])
        #data.append([f"translate {trg_lang} to {src_lang}", trg, src])

    eval_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])

    return train_df, eval_df
    

In [ ]:
%pwd

'/content'

In [ ]:
train_df, eval_df = prepare_translation_datasets(data_path="/content/gdrive/Shareddrives/Gutenberg/MT/corpora", src_lang="english", src="en-CCMatrix_en_ko_clean_5k.txt", trg_lang="korean", trg="ko-CCMatrix_en_ko_clean_5k.txt", test_size=800)

In [ ]:
#show train and eval split
train_df
eval_df

,prefix,input_text,target_text
0,translate english to korean,"'You think they are united, but their hearts a...","그들은 별개의 다른 무리로 보이지만, 그들의 꼬리는 함께 묶여 있다.”라고 했다."
1,translate english to korean,There are some job opportunities that hire peo...,몇 가지 직업 기회가 내 여름 차에 당신 같은 사람을 고용 하 고 있습니다.
2,translate english to korean,[Read: Are you jealous of your ex and their ne...,[읽기 : 당신은 당신의 전과 그들의 새로운 삶을 질투합니까?]
3,translate english to korean,"And, besides, the owner lives in Naples and we…""","그리고, 그 주인이 나폴리에 살아서 우리는....”"
4,translate english to korean,"So, you have very good timing information at t...","그래서, 당신은 위성 수준에서 매우 좋은 타이밍 정보를 가지게 된다."
...,...,...,...
795,translate english to korean,"You need to talk more, what she!","당신 이야기 할 필요가 더, 그녀가 무엇!"
796,translate english to korean,"Today, we pride ourselves on working only with...","오늘날, 우리는 여러 대륙에있는 최고의 데이터 센터에서만 일하는 것을 자랑스럽게 생..."
797,translate english to korean,"But in the first minute, foreign camera girls ...","그러나 첫 분, 외국 카메라 여자가 당신을 볼 수 없습니다."
798,translate english to korean,"""Other people and things may stop you temporar...","""다른 사람들과 사물들이 당신을 일시적으로 멈추게 할 수 있습니다."


Train mt5 

In [ ]:
#create folder named data
# train_df.to_csv("data/train.tsv", sep="\t")
# eval_df.to_csv("data/eval.tsv", sep="\t")

import logging
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MT5Model, MT5Config
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
# train_df = pd.read_csv("data/train.tsv", sep="\t").astype(str)
# eval_df = pd.read_csv("data/eval.tsv", sep="\t").astype(str)

#train_df["prefix"] = ""
#eval_df["prefix"] = ""
model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 10
model_args.eval_batch_size = 10
model_args.num_train_epochs = 1           # This should probably be increased
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
#model_args.wandb_project = "MT5 English-Korean Translation"


# tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
# model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

model = T5Model("mt5", "google/mt5-small", args=model_args,use_cuda=True)

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

In [ ]:
!pip install wandb
import wandb
# using wandb throws a Network connection error
wandb.init(settings=wandb.Settings(start_method="thread"))

In [ ]:
# Train the model
#might cause OSError if cdir not pointing to /content; cant create a dir in gdrive
model.train_model(train_df, eval_data=eval_df)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/4200 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3557: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_model: Training started


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/420 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/800 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model: Training of google/mt5-small model complete. Saved to outputs/.


(420,
 {'eval_loss': [3.1129952520132065],
  'global_step': [420],
  'train_loss': [3.79583740234375]})

In [ ]:
#trying with opennmt update vocab
# need to initalize the class Runner
def update_vocab(self, output_dir, src_vocab=None, tgt_vocab=None):
        """Updates model vocabularies.
        Args:
          output_dir: Directory where the update checkpoint will be saved.
          src_vocab: Path to the new source vocabulary.
          tgt_vocab: Path to the new tagret vocabulary.
        Returns:
          Path to the new checkpoint directory.
        """
        if not isinstance(self._model, models.SequenceToSequence):
            raise ValueError(
                "Updating vocabularies is only supported for sequence to sequence models"
            )
        config = self._finalize_config()
        if src_vocab is None and tgt_vocab is None:
            return config["model_dir"]

        model = self._init_model(config)
        optimizer = model.get_optimizer()
        cur_checkpoint = checkpoint_util.Checkpoint.from_config(
            config, model, optimizer=optimizer
        )
        cur_checkpoint.restore()
        model.create_variables(optimizer=optimizer)
        source_dir = self.model_dir

        self._config["model_dir"] = output_dir
        if src_vocab is not None:
            self._config["data"]["source_vocabulary"] = src_vocab
        if tgt_vocab is not None:
            self._config["data"]["target_vocabulary"] = tgt_vocab
        new_config = self._finalize_config()
        new_model = self._init_model(new_config)
        new_optimizer = new_model.get_optimizer()
        new_checkpoint = checkpoint_util.Checkpoint.from_config(
            new_config, new_model, optimizer=new_optimizer
        )
        new_model.create_variables(optimizer=new_optimizer)

        model.transfer_weights(
            new_model, new_optimizer=new_optimizer, optimizer=optimizer
        )
        new_optimizer.iterations.assign(optimizer.iterations)
        new_checkpoint.save()
        _forward_model_description(source_dir, output_dir)
        return output_dir


dir=update_vocab(t5,output_dir='./content/opennmt',src_vocab='./content/opennmt',tgt_vocab='./content/opennmt')

AttributeError: ignored

In [ ]:
%pwd


In [ ]:
%cd /content


/content


In [ ]:
%%writefile step2.py
##### SCRIPT STARTS HERE #####
#!usr/bin/bash python
import os
import argparse
from tqdm import tqdm
from typing import List
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import RandomSampler
from transformers import (
    MT5Model,
    MT5Config,
    AdamW,
    get_linear_schedule_with_warmup,
)
import sentencepiece as spm

from data_utils import wikiADataset
from common import check_dir, mask_tokens


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--gpu", type=str, default="1")
    parser.add_argument(
        "--dataset_pth", type=str, default="./data/arb-CCAligned_arb_en_clean.txt "
    )
    parser.add_argument(
        "--snp_path", type=str, default="./arb.model"
    )  # sentencepiece model path
    parser.add_argument("--vocab_size", type=int, default=32000)
    parser.add_argument(
        "--save_path", type=str, default="./models/mt5_arb-en/"
    )
    parser.add_argument("--model_name", type=str, default="mt5")
    return parser.parse_args()


def collate(examples: List[torch.Tensor]):
    if sp.pad_id() is None:
        return pad_sequence(examples, batch_first=True)
    return pad_sequence(examples, batch_first=True, padding_value=sp.pad_id())


def train_model(model, train_dataloader, optimizer, scheduler):
    print(args.save_path, args.save_path + f"{args.model_name}/")
    check_dir(args.save_path)
    check_dir(args.save_path + f"{args.model_name}/")

    model = model.train()
    model.zero_grad()

    tr_loss = []
    for batch in tqdm(train_dataloader):
        inputs, labels = mask_tokens(batch)
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        inputs = inputs.to(device)
        labels = labels.to(device)
        # outputs = model(inputs, masked_lm_labels=labels)

        outputs = model(inputs)
        loss = outputs[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)

        tr_loss.append(loss.item())
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    with open(args.save_path + f"{args.model_name}/train_loss.txt", "w") as f:
        for item in tr_loss:
            f.write("%s\n" % item)

    model.save_pretrained(args.save_path + f"{args.model_name}")
    # torch.save(model, args.save_path + f'{args.model_name}.model')

def main():
    global args, sp
    args = parse_args()
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

    sp = spm.SentencePieceProcessor()
    sp.load(args.snp_path)

    wiki_dataset = wikiADataset(sp, args.dataset_pth)
    train_sampler = RandomSampler(wiki_dataset)

    train_dataloader = DataLoader(
        wiki_dataset, sampler=train_sampler, batch_size=64, collate_fn=collate
    )

    conf = MT5Config()  # import 'standard' config from HF
    conf.num_hidden_layers = 8
    conf.vocab_size = args.vocab_size

    model = MT5Model.from_pretrained("google/mt5-small")
    tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    # model = model.to('cuda')

    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=1e-4, eps=1e-8)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)
    )
    # train first level model
    train_model(model, train_dataloader, optimizer, scheduler)


if __name__ == "__main__":
    main()

Writing step2.py


In [ ]:
!python3 step2.py --dataset_pth=/content/gdrive/MyDrive/data/arb-CCAligned_arb_en_clean.txt --snp_path=./models/snp/arbCCAligned.model --vocab_size=32000  --save_path=./models/rawBert_rawTokenizer/ --model_name='Bert'

Traceback (most recent call last):
  File "step2.py", line 133, in <module>
    main()
  File "step2.py", line 87, in main
    sp.load(args.snp_path)
  File "/usr/local/lib/python3.7/dist-packages/sentencepiece/__init__.py", line 367, in Load
    return self.LoadFromFile(model_file)
  File "/usr/local/lib/python3.7/dist-packages/sentencepiece/__init__.py", line 171, in LoadFromFile
    return _sentencepiece.SentencePieceProcessor_LoadFromFile(self, arg)
OSError: Not found: "./models/snp/arbCCAligned.model": No such file or directory Error #2


In [ ]:
%%writefile step3.py
##### SCRIPT STARTS HERE #####
#!usr/bin/bash python
import argparse


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--wiki_vocab', type=str, default='./models/snp/wikiA8k.vocab')
    parser.add_argument('--task_vocab', type=str, default='./models/snp/Quora8k.vocab')
    parser.add_argument('--out_vocab', type=str, default="./models/matcher/wiki_quora_match")
    parser.add_argument('--matcher', type=int, default=1)
    return parser.parse_args()

#does some sort of indexing on the vocabs before mathcing them for overlaps
def update_dict(vocab_pth):
    with open(vocab_pth, 'r') as f: #, encoding="utf-8",errors='ignore'
        output = f.readlines()

    o = [output[i] for i in range(0, 7)]
    idx = 7
    for row in output[7:]:
        s = row.split('\t')[0]
        s += "\t-" + str(idx) + '\n'
        o.append(s)
        idx += 1
    return o


def match(s, dict16):
    res = []
    if s in dict16: return [[s, ], ]
    for idx in range(len(s) - 1):
        if s[:idx] in dict16:
            pieces = match(s[idx:], dict16)
            if pieces:
                for p in pieces:
                    res.append([s[:idx], ] + p)
    return res


def matcher(wiki_voc, sent_voc):
    dict16 = dict()
    for line in wiki_voc:
        tok16, id16 = line.strip().split('\t')
        dict16[tok16] = id16
    out = []
    for line in sent_voc:
        tok64, id64 = line.strip().split('\t')
        if tok64 in dict16:
            out.append("\t".join(map(str, [id64, tok64, dict16[tok64], tok64])))
            pass
        else:
            r = match(tok64, dict16)
            if r:
                if args.matcher == 1:
                    optimal = list(sorted(r, key=lambda x: -max(map(len, x)) * 1000 + len(x)))[0]
                    toks = []
                    ids = []
                    for q in optimal:
                        toks.append(q)
                        ids.append(dict16[q])
                    out.append("\t".join(map(str, [id64, tok64, ','.join(ids), ','.join(toks)])))
                elif args.matcher == 2:
                    if len(r) > 1:
                        ml = min(map(len, r))
                        r = list(filter(lambda x: len(x) == ml, r))
                    toks = ";".join(map(lambda x: ",".join(x), r))
                    ids = ";".join(map(lambda x: ",".join(map(lambda a: dict16[a], x)), r))
                    out.append("\t".join(map(str, [id64, tok64, ids, toks])))
            else:
                out.append("\t".join(map(str, [id64, tok64, '--unknown--'])))

    with open(f'{args.out_vocab}_matcher_f{args.matcher}.tsv', 'w') as f:
        for item in out:
            f.write("%s\n" % item)


def main():
    global args
    args = parse_args()

    sent_voc = update_dict(args.task_vocab)
    wiki_voc = update_dict(args.wiki_vocab)
    
    matcher(wiki_voc, sent_voc)


if __name__ == '__main__':
    main()


Overwriting step3.py


In [ ]:
!python3 step3.py --wiki_vocab=/content/en-CCMatrix.vocab --task_vocab=/content/oro-HARX.vocab --out_vocab=./matcher/en_oro_match --matcher=1

In [ ]:
!python3 step3.py --wiki_vocab=/content/en-CCMatrix.vocab --task_vocab=/content/oro-HARX.vocab --out_vocab=./matcher/en_oro_match --matcher=2

In [ ]:
!python3 step3.py --wiki_vocab=/content/ko-CCMatrix.vocab --task_vocab=/content/ko-BTKX.vocab --out_vocab=./matcher/ko_ko_match --matcher=1

In [ ]:
!python3 step3.py --wiki_vocab=/content/ko-CCMatrix.vocab --task_vocab=/content/ko-BTKX.vocab --out_vocab=./matcher/ko_ko_match --matcher=2

Step 4. step 4 - Transfer dictionary using mapping. Сreate folders and raw models for experiments.

In [ ]:

%%writefile step4.py
##### SCRIPT STARTS HERE #####
#!usr/bin/bash python
import argparse
import torch
from transformers.utils.dummy_pt_objects import MT5ForConditionalGeneration
from transformers import T5Model, T5Config, T5ForConditionalGeneration
#https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html
from transformers import BertConfig, BertForSequenceClassification

from common import check_dir


def get_pretrained_model(model_dir):
    model= T5Model.from_pretrained(model_dir)
    return model

# ##Alt 1
# def get_random_embeds(config,new_vocab_size):
#    #get 'random' set of embeddings from a pretrained mt5

#     random_model = T5ForConditionalGeneration(config)
#     random_embeds= random_model.shared.embeddings.word_embeddings.weight

##Alt2
def get_random_embeds(new_vocab_size,model):
    # weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
    random_embeds = torch.nn.Embedding(new_vocab_size,model.shared.embedding_dim)
    random_embeds.weight.data = model.shared.weight.data
    #random_embeds = torch.nn.Embedding.from_pretrained(new_vocab_size ,model.shared.embedding_dim)
    return random_embeds

    ##original
# def get_random_embeds(config):
#     random_model = BertForSequenceClassification(config)
#     random_embeds = random_model.bert.embeddings.word_embeddings.weight
#     return random_embeds





def get_mapping_matrices(mapping_file, new_vocab_size, old_vocab_size, embedding_dim=768, use_bad_shift=False,
                         use_one_to_one=False):
  #compare old and new vocab
    mapping_matrix = torch.zeros((new_vocab_size, old_vocab_size))
    mask_matrix = torch.zeros((new_vocab_size, embedding_dim))

    with open(mapping_file) as f:
        for _ in range(7):
            f.readline()  # skip first 7 lines

        for line in f:
            fields = line.rstrip().split('\t')
            if fields[-1] == '--unknown--':
                continue
            # only transfer wholesome old token embeds if use_one_to_one
            if ',' in fields[2] and use_one_to_one:
                continue

            new_idx = -int(fields[0])
            mask_matrix[new_idx, :] = 1

            old_ids_variants = fields[2].split(';')
            denominator = float(len(old_ids_variants))

            for old_ids in old_ids_variants:
                old_ids = old_ids.split(',')

                # replace embeds with bad ones if use_bad_shift
                shift = 3 if use_bad_shift and len(old_ids) > 1 else 0

                old_ids = [min(old_vocab_size, -int(idx) + shift) for idx in old_ids]
                local_denominator = len(old_ids)
                for idx in old_ids:
                    mapping_matrix[new_idx, idx] += 1. / local_denominator

            mapping_matrix[new_idx, :] /= denominator
    return (mapping_matrix, mask_matrix)


def create_experiment(mapping_file, old_vocab_size, new_vocab_size, use_bad_shift, use_one_to_one, save_pht, name):
    conf = T5Config()
    conf.vocab_size = new_vocab_size
    #conf.num_hidden_layers = 8
    model = get_pretrained_model(args.source_bert_model); #points to pre-trained parent model
    random_embeds = get_random_embeds(new_vocab_size,model)
    mapping_matrix, mask_matrix = get_mapping_matrices(mapping_file,
                                                       new_vocab_size,
                                                       old_vocab_size,
                                                       use_bad_shift=use_bad_shift,
                                                       use_one_to_one=use_one_to_one)
    new_embeds = mapping_matrix.matmul(model.shared.weight.data)
    #new_embeds = mapping_matrix.matmul(model.shared.embeddings.word_embeddings.weight) #take embeddings from pretrained
    new_embed_matrix = (1. - mask_matrix) * random_embeds + new_embeds
    model.shared.embeddings.word_embeddings = torch.nn.Embedding.from_pretrained(new_embed_matrix, freeze=False)
    check_dir(save_pht + name)
    model.save_pretrained(save_pht + name)

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--mapping_file_1', type=str, default='./models/matcher/wiki_quora_match_matcher_f1.tsv')
    parser.add_argument('--mapping_file_2', type=str, default='./models/matcher/wiki_quora_match_matcher_f2.tsv')
    parser.add_argument('--source_bert_model', type=str, default='./models/rawBert_rawTokenizer/Bert8_WikiData_tok8k/')
    parser.add_argument('--new_vocab_size', type=int, default=8000)
    parser.add_argument('--old_vocab_size', type=int, default=8000)

    parser.add_argument('--save_pht', type=str, default='./models/matched_models/ex1/')

    parser.add_argument('--use_random_embeds', type=bool, default=False)
    parser.add_argument('--use_bad_shift', type=bool, default=False)
    parser.add_argument('--use_one_to_one', type=bool, default=False)

    return parser.parse_args()


def main():
    global args
    args = parse_args()

    check_dir(args.save_pht)

    create_experiment(args.mapping_file_2, args.old_vocab_size, args.new_vocab_size, use_one_to_one=False,
                      use_bad_shift=False, save_pht=args.save_pht, name='avg')

    create_experiment(args.mapping_file_2, args.old_vocab_size, args.new_vocab_size, use_one_to_one=False,
                      use_bad_shift=False, save_pht=args.save_pht, name='avgavg')

    create_experiment(args.mapping_file_1, args.old_vocab_size, args.new_vocab_size, use_one_to_one=True,
                      use_bad_shift=False, save_pht=args.save_pht, name='matched')

    create_experiment(args.mapping_file_1, args.old_vocab_size, args.new_vocab_size, use_one_to_one=False,
                      use_bad_shift=True, save_pht=args.save_pht, name='shifted')

    create_experiment(args.mapping_file_2, args.old_vocab_size, args.new_vocab_size, use_one_to_one=False,
                      use_bad_shift=True, save_pht=args.save_pht, name='shifted2')

    conf = T5Config()
    conf.vocab_size = args.new_vocab_size
    #conf.num_hidden_layers = 8


    model = get_pretrained_model(args.source_bert_model);
    check_dir(args.save_pht + 'pretrain')
    model.save_pretrained(args.save_pht + 'pretrain')

    #random_embeds = get_random_embeds(conf)
    # model.bert.embeddings.word_embeddings = torch.nn.Embedding.from_pretrained(random_embeds, freeze=False)
    # check_dir(args.save_pht + 'randomhead')
    # model.save_pretrained(args.save_pht + 'randomhead')

    random_model = MT5Model(conf)
    check_dir(args.save_pht + 'random')
    random_model.save_pretrained(args.save_pht + 'random')

if __name__ == '__main__':
    main()

Overwriting step4.py


In [ ]:
!python3 step4.py --mapping_file_1=./matcher/en_oro_match_matcher_f1.tsv --mapping_file_2=./matcher/en_oro_match_matcher_f1.tsv --source_bert_model=/content/outputs/best_model --new_vocab_size=8000 --old_vocab_size=250112 --save_pht=./childmodels/matched_models

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /content/outputs/best_model were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
tcmalloc: large alloc 8003584000 bytes == 0x7a0b0000 @  0x7fd3a79c6b6b 0x7fd3a79e6379 0x7fd2ede0750e 0x7fd2eddf97c2 0x7fd327d6b10f 0x7fd327d6ba51 0x7fd327d6baa4 0x7fd32823e9ae 0x7fd32892baea 0x7fd3286ed38c 0x7fd32891262f 0x7fd3287281e0 0x7fd3a34f7371 0x7fd328242cf0 0x7fd328ac3

In [ ]:
%pwd

'/content'

In [ ]:
%%writefile step5.py
##### SCRIPT STARTS HERE #####
#!usr/bin/bash python

import argparse
import os
from typing import List
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import RandomSampler
from torch.nn.utils.rnn import pad_sequence
from simpletransformers.t5 import T5Model, T5Args
from transformers import get_linear_schedule_with_warmup, AdamW
import sentencepiece as spm

from data_utils import QuoraMLMDataset
from common import check_dir, mask_tokens

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--use_gpu', default="0")
    parser.add_argument('--experiment_folder', required=True)  # default='random/'
    parser.add_argument('--sp_model_pth', default='./models/snp/Quora8k.model')
    parser.add_argument('--experiments_dir', default='quora_ex1')
    return parser.parse_args()


def main():
    args = parse_args()
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = args.use_gpu

    ROOT_DIR = './output/models/matched_models/'
    PRETRAIN_MODEL_PTH = os.path.join(ROOT_DIR, args.experiments_dir, args.experiment_folder)
    SAVE_MODEL_PHT = os.path.join(ROOT_DIR, args.experiments_dir+"_2mlm", args.experiment_folder)
    check_dir(os.path.join(ROOT_DIR, args.experiments_dir+"_2mlm"))
    check_dir(SAVE_MODEL_PHT)

    model = MT5Model.from_pretrained(PRETRAIN_MODEL_PTH)
    model.to(device='cuda')
    print('model load', PRETRAIN_MODEL_PTH)

    sp = spm.SentencePieceProcessor()
    sp.load(args.sp_model_pth)
    print('SentencePiece size', sp.get_piece_size(), args.sp_model_pth)

    def collate(examples: List[torch.Tensor]):
        if sp.pad_id() is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=sp.pad_id())

    dataset = QuoraMLMDataset(sp)
    train_sampler = RandomSampler(dataset)
    train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=64, collate_fn=collate)

    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4, eps=1e-8)

    t_total = len(train_dataloader)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=t_total)

    model.train()
    model.zero_grad()
    tr_loss = []
    for batch in tqdm(train_dataloader):
        inputs, labels = mask_tokens(batch)
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        outputs = model(inputs, masked_lm_labels=labels)
        loss = outputs[0]

        loss.backward();
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)

        tr_loss.append(loss.item())
        optimizer.step();
        scheduler.step()
        model.zero_grad();

    with open(SAVE_MODEL_PHT + '/step_mlm_batch_loss.txt', 'w') as f:
        for item in tr_loss:
            f.write("%s\n" % item)

    model.save_pretrained(SAVE_MODEL_PHT)


if __name__ == '__main__':
    main()


In [ ]:
#!/bin/bash
!EXP=('avg/' 'avgavg/' 'matched/' 'pretrain/' 'randomhead/' 'shifted/' 'shifted2/','random')

!for exp in "${EXP[@]}"
!do
!  echo "Start $exp please wait ..."
!  python3 step5.py --experiments_dir=quora_ex1 --experiment_folder="$exp" --sp_model_pth=./models/snp/Quora8k.model
!done